# Imports

In [40]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.fft import rfft, rfftfreq
from sklearn.metrics import accuracy_score
import librosa
import resampy
import librosa.display
import IPython.display as ipd
import os
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pydub import AudioSegment
import os
warnings.filterwarnings('ignore')


# Model

### define features extraction function

In [41]:
def mfcc_feature_extractor(audio,sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T,axis=0)
    
    return mfccsScaledFeatures

In [42]:
def contrast_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate) 
    contrastScaled = np.mean(contrast.T,axis=0)
    
    return contrastScaled

In [43]:
def tonnetz_feature_extractor(audio,sampleRate):
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T,axis=0)
    
    return tonnetzScaled

In [44]:
def centroid_feature_extractor(audio,sampleRate):
    centroid = librosa.feature.spectral_centroid(y=audio,sr=sampleRate) 
    centroidScaled = np.mean(centroid.T,axis=0)
    
    return centroidScaled

In [45]:
def chroma_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    chroma = librosa.feature.chroma_stft(S=stft,sr=sampleRate)
    chromaScaled = np.mean(chroma.T,axis=0)
    
    return chromaScaled

In [46]:
def features_extractor(file):
    if file.lower().endswith(".m4a"):
        audio = AudioSegment.from_file(file, format="m4a")
        file_name, _ = os.path.splitext(os.path.basename(file))
        output_path = os.path.join(os.path.dirname(file), file_name + ".wav")
        audio.export(output_path, format="wav")
        # os.remove(file)
        file = output_path

    features = []
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfcc = mfcc_feature_extractor(audio, sample_rate)
    contrast = contrast_feature_extractor(audio, sample_rate)
    tonnetz = tonnetz_feature_extractor(audio, sample_rate)
    chroma = chroma_feature_extractor(audio, sample_rate)

    # Concatenate individual feature arrays
    concatenated_features = np.concatenate((mfcc, contrast, tonnetz, chroma), axis=0)
    features.append(concatenated_features)

    return features

### define add features function

In [47]:
def add_features(extractedFeatures, dirPath, label):
    for file in os.listdir(dirPath):
        filePath = os.path.join(dirPath, file)

        if filePath.lower().endswith(".m4a"):
            audio = AudioSegment.from_file(filePath, format="m4a")
            file_name, _ = os.path.splitext(os.path.basename(filePath))
            output_path = os.path.join(os.path.dirname(filePath), file_name + ".wav")
            audio.export(output_path, format="wav")
            # os.remove(filePath)
            filePath = output_path

        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc = mfcc_feature_extractor(audio, sampleRate)
        contrast = contrast_feature_extractor(audio, sampleRate)
        tonnetz = tonnetz_feature_extractor(audio, sampleRate)
        chroma = chroma_feature_extractor(audio, sampleRate)

        extractedFeatures.append([mfcc, contrast, tonnetz, chroma, label])

### creating dictionary with dirPath and label

In [48]:
dict1 = {
    "ibrahim": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ibrahim",
    ],
    "ahmeda": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmeda",
    ],
    "omar": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar",
    ],
    "hazem": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem",
    ],
    "ahmedk": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmedk",
    ],
    "hassan": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hassan",
    ],
    "mohannad": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/mohannad",
    ],
    "other":[
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people", 
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer", 
    ],
    "abdelrahman": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/abdelrahman",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/abdelrahman",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/abdelrahman",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/abdelrahman",
    ],
}

### creating data frame from our data

In [49]:
extractedFeatures = []
for label in dict1.keys():
    for link in dict1[label]:
        add_features(extractedFeatures, link, label)

In [50]:
data = pd.DataFrame(extractedFeatures, columns=['mfcc','contrast','tonnetz','chroma','class'])
data.to_csv('processing.csv')
print(data.shape)
data.describe().T

(614, 5)


,count,unique,top,freq
mfcc,614,614,"[-516.205, 153.0361, -8.050304, 33.207005, 1.4...",1
contrast,614,614,"[21.502896924279796, 15.580563903506814, 14.88...",1
tonnetz,614,614,"[0.05704616298945937, 0.08094122501686858, 0.1...",1
chroma,614,614,"[0.48283857, 0.58331424, 0.63629895, 0.4935803...",1
class,614,9,other,169


### splitting the data into features and target

In [51]:
features = data.iloc[:,0:-1]
target = data['class']
target.unique()

array(['ibrahim', 'ahmeda', 'omar', 'hazem', 'ahmedk', 'hassan',
       'mohannad', 'other', 'abdelrahman'], dtype=object)

In [52]:
features = features.values.tolist()
for i in range(len(features)):
    features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))

### encoding the target

In [53]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)

### splitting the data into train and test

In [54]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.25, random_state=29)

In [55]:
for i in range(0,8,1):
  print(i,"  ", len(target[target==i]))
  # {'ahmeda': 0, 'ahmedk': 1, 'hassan': 2, 'hazem': 3, 'ibrahim': 4, 'mohannad': 5, 'omar': 6, 'other': 7}

0    45
1    34
2    60
3    32
4    79
5    80
6    33
7    82


### creating SVM model

In [56]:
# classifier = SVC(kernel='linear',decision_function_shape="ovo" ,probability=True) 
# classifier.fit(xTrain,yTrain)

### creating XGBoost model

In [57]:
classifier = XGBClassifier(objective='multi:softmax', num_class=8, eval_metric='mlogloss')
classifier.fit(xTrain, yTrain)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=8, num_parallel_tree=None, ...)

### creating RandomForest model

In [58]:
# classifier = RandomForestClassifier(n_estimators=100, random_state=42)  
# classifier.fit(xTrain, yTrain)

### predicting xTest

In [59]:
predX = classifier.predict(xTest)

### making classification report to see the result of the predicton 

In [60]:
print(classification_report(yTest, predX))

train_predictions = classifier.predict(xTrain)
test_predictions = classifier.predict(xTest)

train_accuracy = accuracy_score(yTrain, train_predictions)
test_accuracy = accuracy_score(yTest, test_predictions)

print(train_accuracy, "  ", test_accuracy)

              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       0.86      1.00      0.92        12
           2       1.00      1.00      1.00        13
           3       0.86      1.00      0.92         6
           4       0.94      0.94      0.94        17
           5       0.90      0.95      0.93        20
           6       1.00      0.70      0.82        10
           7       0.94      1.00      0.97        17
           8       1.00      0.98      0.99        50

    accuracy                           0.95       154
   macro avg       0.94      0.94      0.94       154
weighted avg       0.96      0.95      0.95       154

1.0    0.9545454545454546


In [78]:
def prediction(pred):   
    # {'ahmeda': 0, 'ahmedk': 1, 'hassan': 2, 'hazem': 3, 'ibrahim': 4, 'mohannad': 5, 'omar': 6, 'other': 7}
    if pred == 0:
        return ("abdelrahman")
    elif pred == 1:
        return ("ahmeda")
    elif pred == 2:
        return ("ahmedk")
    elif pred == 3:
        return ("hassan")
    elif pred == 4:
        return ("hazem")
    elif pred == 5:
        return ("ibrahim")
    elif pred == 6:
        return ("mohannad")
    elif pred == 7:
        return ("omar")
    elif pred == 8:
        return ("other")

In [79]:
def my_predict(features):
    probabilities = classifier.predict_proba(features)
    # print(probabilities)

    for i in range(len(features)):
        class_pred_index = probabilities[i].argmax()  
        class_pred = probabilities[i, class_pred_index]  
        threshold = 0.75
        
        if class_pred > threshold:
            return class_pred_index
        else:
            return 7

In [80]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmedk")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmedk/{file}") 
  print(classifier.predict_proba(testFeatures))
  print("1: ",prediction(classifier.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[4.4149844e-04 2.7226814e-04 9.9621528e-01 3.8169583e-04 4.2453525e-04
  3.6370009e-04 2.8607543e-04 4.6650853e-04 1.1485552e-03]]
[2]
1:  ahmedk
2:  ahmedk
[[1.0991342e-03 6.7782705e-04 9.9183196e-01 7.0229330e-04 1.4033066e-03
  7.5375725e-04 7.1220053e-04 9.0374728e-04 1.9157856e-03]]
[2]
1:  ahmedk
2:  ahmedk
[[5.9992290e-04 3.6996731e-04 9.9526608e-01 3.8332134e-04 6.3969620e-04
  4.4833357e-04 3.8872892e-04 6.3390745e-04 1.2700156e-03]]
[2]
1:  ahmedk
2:  ahmedk
[[3.9731080e-04 2.4501828e-04 9.9665987e-01 2.5386215e-04 3.2410931e-04
  5.2853557e-04 2.5744367e-04 4.4698021e-04 8.8697480e-04]]
[2]
1:  ahmedk
2:  ahmedk
[[3.1940872e-04 2.7693395e-04 9.9374145e-01 2.8693004e-04 3.4587385e-04
  5.1744533e-04 2.9097783e-04 5.5828242e-04 3.6626994e-03]]
[2]
1:  ahmedk
2:  ahmedk


In [81]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/omar")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/omar/{file}") 
  print(classifier.predict_proba(testFeatures)*100)
  print("1: ",prediction(classifier.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[ 2.7488594  1.58241    3.8227704  2.0380929  4.7362075  3.8146112
   1.3866773 78.27659    1.5937811]]
1:  omar
2:  omar
[[ 2.7488594  1.58241    3.8227704  2.0380929  4.7362075  3.8146112
   1.3866773 78.27659    1.5937811]]
1:  omar
2:  omar
[[ 6.9359245  2.8084505  4.895044   1.6950024  4.728669   4.0906725
   1.9899137 70.01802    2.8383102]]
1:  omar
2:  omar
[[ 6.9359245  2.8084505  4.895044   1.6950024  4.728669   4.0906725
   1.9899137 70.01802    2.8383102]]
1:  omar
2:  omar
[[ 0.97868246  0.16720338  0.50934976  0.3619991   0.17993034  0.27745456
   0.20843662 97.05155     0.26539516]]
1:  omar
2:  omar


In [82]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem/{file}") 
  print(classifier.predict_proba(testFeatures)*100)
  print("1: ",prediction(classifier.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

Recording (10)_638396660155783233_wav.wav
[[2.6662689e-02 5.7343412e-02 4.3690387e-02 9.0203173e-02 9.7874443e+01
  8.6081259e-02 3.2052826e-02 6.0522139e-01 1.1842983e+00]]
1:  hazem
2:  hazem
Recording (10)_638397160016988114_wav.wav
[[ 0.40198332  0.34320405  0.18178555  0.14250684 98.31476     0.09997299
   0.13540852  0.15448573  0.22589397]]
1:  hazem
2:  hazem
Recording (11)_638396660155986766_wav.wav
[[ 0.9058727   0.2394497   0.23543239  0.17220719 93.50446     0.14909138
   0.17272149  1.095186    3.5255744 ]]
1:  hazem
2:  hazem
Recording (11)_638397160017160202_wav.wav
[[2.12465972e-01 1.74694493e-01 1.11789025e-01 9.84428450e-02
  9.85966644e+01 4.78911400e-01 1.56728625e-01 1.21430345e-01
  4.88762558e-02]]
1:  hazem
2:  hazem
Recording (12)_638396660156118051_wav.wav
[[ 0.22220309  0.27284932  0.4684556   0.38005254 95.803474    0.22109577
   0.12433653  1.7022731   0.8052634 ]]
1:  hazem
2:  hazem


In [83]:
testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/dataset/recorded/1.wav") 
print(classifier.predict_proba(testFeatures)*100)
print("1: ",prediction(classifier.predict(testFeatures)))
print("2: ",prediction(my_predict(testFeatures))) 

[[ 1.5854694   0.43981007  0.60611767  0.6622817   5.079853    0.25413528
   0.36800468  3.9580405  87.04629   ]]
1:  other
2:  other


In [84]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer/{file}") 
  print(classifier.predict_proba(testFeatures)*100)
  print("1: ",prediction(classifier.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[6.2036552e-03 5.4373629e-03 1.5039996e-02 5.4222266e-03 1.8205790e-02
  1.2029541e-02 3.8247136e-03 4.9267490e-03 9.9928902e+01]]
1:  other
2:  other
[[7.5637153e-03 8.5005052e-03 2.6504014e-02 1.0347673e-02 3.1621933e-02
  3.7527628e-02 7.2990181e-03 1.1022901e-02 9.9859612e+01]]
1:  other
2:  other
[[5.8493060e-03 9.3041277e-03 2.2051904e-02 9.2687737e-03 2.8733334e-02
  2.4155870e-02 6.5379799e-03 8.3235828e-03 9.9885773e+01]]
1:  other
2:  other
[[1.3255341e-02 1.5598810e-02 5.5521924e-02 1.3886683e-02 9.1019481e-02
  4.6477698e-02 1.3394030e-02 3.9849199e-02 9.9710999e+01]]
1:  other
2:  other
[[4.4741090e-03 4.1721426e-03 1.6793078e-02 4.4763410e-03 2.6919436e-02
  1.7957728e-02 4.3175342e-03 1.2911025e-02 9.9907982e+01]]
1:  other
2:  other


In [85]:
pickle_out = open("processing.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()